# Q2 Implementing a Decision Tree (35 points)

Now, you will be implementing a decision tree predicting people's well-being as you did in Q1. Make sure you finished [Q1](./nn.ipynb) before starting on this as there will be questions asking you to reuse the code there.

Let's start by importing the neccessary modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from treeart import binary_edge
%matplotlib inline

Load the data from folder `/dataset`

In [ ]:
file_path = './datasets'

# If you are running from Google Colab, uncomment the section below
# # Install treeart (The only module that isn't already installed on Colab)
# !pip install treeart
# # Mount google drive to allow access to your files
# from google.colab import drive
# drive.mount('/content/drive')
# drive_folder = '/content/drive/MyDrive'

# # Ajust this line to be the assignment2 folder in your google drive
# notebook_folder = drive_folder + '/assignment2'
# file_path = notebook_folder + file_path

In [ ]:
df_train = pd.read_csv(file_path + '/train.csv')

df_train.info()

Split the training data into `x_train` and `y_train`

In [ ]:
# Target variable (y)
df_train_y = df_train[['Dominant_Emotion']]

# Features (X)
df_train_x = df_train.drop('Dominant_Emotion', axis=1)

print(df_train_x.shape)
print(df_train_y.shape)

## Data Preprocessing

### 1. Remove unnecessary features

In [ ]:
## TODO: Remove any feature that seems to not have much impact on our model's prediction

# Display all columns
df_train_x.info()

### 2. Filling Missing Entries

Please copy the `handle_null` function in Q1.1 and paste below

In [ ]:
%%capture --no-stdout

def handle_null(train_x, train_y):
    # TODO: Copy the entire preprocess function below

    # TODO: Handle missing entries for the remaining features in the training set

    # TODO: Handle missing entries for the "Dominant_Emotion" feature in the test set
    pass

# Preprocess and display data
handle_null(df_train_x, df_train_y)

null_counts_x = pd.DataFrame({'Types': df_train_x.dtypes, 'Null Values': df_train_x.isnull().sum()})
null_counts_y = pd.DataFrame({'Types': df_train_y.dtypes, 'Null Values': df_train_y.isnull().sum()})

print("Training set:")
print(null_counts_x)
print("\nTest set:")
print(null_counts_y)

#### Encoding

Since decision trees can categorize labels independently without adding any bias, it is generally not necessary to apply one hot encoding to the label in this case. Therefore, we will just need to apply one hot encoding to the input features only.

In [ ]:
df_train_x = pd.get_dummies(df_train_x, columns=['Gender', 'Platform'], dtype=int)

label_encoder = {}
text_labels = np.unique(df_train_y['Dominant_Emotion'].to_numpy())
for i, label in enumerate(text_labels):
    label_encoder[label] = i

df_train_y = df_train_y.replace(label_encoder)

df_train_x.info(), df_train_y.info()

## Tree Implementation

For implementing a decision tree, we use classes for nodes and tree so that (hopefully!) you can understand it better.

Please take a look at the long code block below and complete the TODOs tasks

In [ ]:
class TreeNode:
    def __init__(self, is_leaf, attr_name, attr_threshold, left, right, label, label_prob):
        """
        Initializes a node in the decision tree. We will use the feature name and a number
        value for the attribute since all features have been converted to be numerical
        through one-hot encoding

        Parameters:
        - is_leaf (bool): Indicates whether the node is a leaf node or not.
        - attr_name (str | None): The name of the attribute used for splitting at this node.
        - attr_threshold (float | None): The threshold value for splitting the attribute.
        - left (TreeNode | None): The left child node for entries with attribute value <= threshold.
        - right (TreeNode | None): The right child node for entries with attribute value > threshold.
        - label (str | None): The label assigned to the leaf node.
        - label_prob (float | None): The probability entries of this label appear in this partition

        Returns:
        - None
        """
        self.is_leaf = is_leaf
        self.attr_name = attr_name
        self.attr_threshold = attr_threshold
        self.left = left
        self.right = right
        self.label = label
        self.label_prob = label_prob

    def classify(self, entry):
        """
        Classifies an entry based on the decision tree.

        Parameters:
        - entry: A dictionary representing the attributes of the entry.

        Returns:
        - label: The predicted label for the entry.
        - label_prob: The probability of the predicted label.

        Note:
        - This method assumes that the decision tree has already been trained.
        """

        # TODO: Add your implementation here
        pass
    

#### Implementing a decision tree

We will build a decision tree using information gain. Here are a few criteria for stopping and returning a leaf node before splitting further:
- All entries in the partition have the same label
- There is no attribute left to split on
- The number of entries in each partition are less than a certain threshold (`min_leaf_count`)

##### Entropy and Information Gain Review

$$
    \begin{align*}
        p_i &= \text{probability that label $i$ occurs in the partition} \\
        I(D) &= -\sum^m_{i=1} p_i \log_2(p_i) \leftarrow \text{entropy of a parition $D$ with $m$ being the number of labels/classes} \\
        \text{Info}_A(D) &= \sum^v_{j=1} \frac{|D_j|}{|D|} I(D_j) \leftarrow \text{average entropy of all smaller partitions from splitting partition $D$ based on attribute $A$} \\
        \text{Gain}_A(D) &= I(D) - \text{Info}_A(D) \leftarrow \text{Information Gain of splitting based on attribute $A$}
    \end{align*}
$$

In [ ]:
class DecisionTree:
    def __init__(self, x_train, y_train, min_leaf_count=10):
        self.x_train = x_train
        self.y_train = y_train
        self.class_name = y_train.columns[0]
        self.class_labels = self.y_train[self.class_name].unique()
        self.class_count = len(self.class_labels)
        self.min_leaf_count = min_leaf_count
        self.root = self.build_tree(x_train)

    ## TODO: From the formulas above, add more functions and implement the DecisionTree class (recursively build the tree)

    def build_tree(self, x_train):
        """
        Builds a decision tree using the training data.

        Returns:
            The root node of the decision tree.
        """
        pass

    def classify(self, entry):
        """Perform inference on a single entry.

        Args:
            entry: the instance being classified

        Returns: a tuple containing a class label and a probability
        """
        return self.root.classify(entry)

    def ascii_tree(self, node):
        """Ultra high tech ascii build maker"""
        if node.is_leaf:
            return "{} = {}".format(text_labels[node.label], node.label_prob)
        attr = "{} (Less <= {} => Greater)".format(node.attr_name, node.attr_threshold)
        left = self.ascii_tree(node.left)
        right = self.ascii_tree(node.right)
        return binary_edge(attr, left, right, align='center')
    def __str__(self):
        return self.ascii_tree(self.root)

#### Tree Visualization

In [ ]:
%%capture --no-stdout

decision_tree = DecisionTree(df_train_x, df_train_y)
print(decision_tree)

## Testing

Now we will now use the model to predict the label of the entries in our test set

In [ ]:
df_test = pd.read_csv(file_path + '/test.csv')

# Split to features and labels
df_test_x = df_test.drop('Dominant_Emotion', axis=1)
df_test_y = df_test[['Dominant_Emotion']]

# Preprocess test dataset
df_test_x = df_test_x.drop('User_ID', axis=1)
handle_null(df_test_x, df_test_y)

df_test_x = pd.get_dummies(df_test_x, columns=['Gender', 'Platform'], dtype=int)
df_test_y = df_test_y.replace(label_encoder)

df_test_x.info(), df_test_y.info()

### Predict and compute accuracy

In [ ]:
predictions = []
confidences = []
for i, entry in df_test_x.iterrows():
    pred, conf = decision_tree.classify(entry)
    predictions.append(pred)
    confidences.append(conf)

print("Confidence: {}".format(np.mean(confidences)))

accuracy = (df_test_y['Dominant_Emotion'] == predictions).mean()
print("Accuracy: {}".format(accuracy))

## Tuning

Now try adjusting different parameters such as `min_leaf_count` or the way thresholds are computed. Describe how accuracies change when you change those parameters

Your answer:

Test out different parameters and try to get the highest accuracy on the test set as possible. Please put in the highest accuracy you got and explain how you achieve it (we will check to see if you're being honest or not 😎)

- Highest accuracy:
- Method: